## Set up the environment

In [2]:
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()
model_package_arn='arn:model-package/merchant-matching'
s3_base_path='s3://[your s3 path]'


## Realtime Inference

In [5]:

from sagemaker import ModelPackage
model_name='merchant-matching-model'
model = ModelPackage(name=model_name,
                     role=role,
                     model_package_arn=model_package_arn,
                     sagemaker_session=sagemaker_session)

In [6]:
endpoint_config_name='merchant-matching-edp-config'
endpoint_name='merchant-matching-edp'

In [7]:
predictor = model.deploy(1, 'ml.p2.xlarge', endpoint_name=endpoint_name)

In [8]:
# connect to the endpoint
predictor = sage.predictor.RealTimePredictor(
    endpoint_name,
    sagemaker_session=sagemaker_session
)

In [9]:
import io
import json
import boto3

client = boto3.client('sagemaker-runtime')

input = [{"requestId": "1","name": "Starbucks, Inc1.","streetAddress": "10 Main St, Apt 3","city": "Los Angeles","state": "CA","postalCode": "10577","country": "USA","phoneNumber": "9175763488"}]

res = client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType="application/json",
                                  Accept="application/json",
                                  Body=json.dumps(input))
body=res['Body'].read()
msg=body.decode('utf-8')
print(msg)
    

In [10]:
# Delete endpoint
predictor.delete_endpoint()

## Batch Transform Job

In [11]:
s3_output_path=s3_base_path + '/output/'
transformer = sage.transformer.Transformer(
    base_transform_job_name='merchant-matching-batch-transform-test',
    model_name=model_name,
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=s3_output_path,
    sagemaker_session=sagemaker_session
)

In [12]:
transformer.transform(
   s3_base_path + '/inputData/'
)

# wait until prediction job is completed
transformer.wait()

#### Inspect the Batch Transform Output in S3